<a href="https://colab.research.google.com/github/HammadN98/pytorch/blob/main/09_pytorch_deploy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 09. Deploy de Modelow

Pra q serve?

Para deixar o codigo que esta no notebook em algo usavel por outros/outra coisa.




## 0. Importando os basicos

* Pacotes
* E baixandos o going_modular do github

In [3]:
#ctrl c + ctrl v dos imports pardrao
# Continue with regular imports
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

# Try to import the going_modular directory, download it from GitHub if it doesn't work
try:
    from going_modular.going_modular import data_setup, engine
    from helper_functions import download_data, set_seeds, plot_loss_curves
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular or helper_functions scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !mv pytorch-deep-learning/helper_functions.py . # get the helper_functions.py script
    !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine
    from helper_functions import download_data, set_seeds, plot_loss_curves

[INFO] Couldn't find torchinfo... installing it.
[INFO] Couldn't find going_modular or helper_functions scripts... downloading them from GitHub.
Cloning into 'pytorch-deep-learning'...
remote: Enumerating objects: 4356, done.
remote: Counting objects: 100% (321/321), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 4356 (delta 213), reused 254 (delta 178), pack-reused 4035 (from 1)
Receiving objects: 100% (4356/4356), 654.51 MiB | 33.25 MiB/s, done.
Resolving deltas: 100% (2585/2585), done.
Updating files: 100% (248/248), done.


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

## 1. Baixando os dados

In [5]:
#ctrl c + ctrl v dos imports pardrao
# Download pizza, steak, sushi images from GitHub
data_20_percent_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip",
                                     destination="pizza_steak_sushi_20_percent")

data_20_percent_path

[INFO] Did not find data/pizza_steak_sushi_20_percent directory, creating one...
[INFO] Downloading pizza_steak_sushi_20_percent.zip from https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip...
[INFO] Unzipping pizza_steak_sushi_20_percent.zip data...


PosixPath('data/pizza_steak_sushi_20_percent')

In [6]:
#ctrl c + ctrl v dos imports pardrao
# Setup directory paths to train and test images
train_dir = data_20_percent_path / "train"
test_dir = data_20_percent_path / "test"

## 2. Experimentos para o Deploy do FoodVisiob Mini

### 3 Perguntas a serem respondidas?

1. QUal o melhor cenario para o deploy desse modeelo de machine learning?
2. Para aondemeu modelo ira?
3. Como o modelo ira funcionar?

**USo ideal do FoodVision Mini:**  Um modelo que performa bem e seeja rapido
  1. PErformace: 95%+ de acurácia
  2. Rapdo: O mais perto de real-time possivel (30FPS+ ou 30ms de latencia)

Para conseguir sses objetivos serao realizados eexperimentos em dois modelos:
  1. EffNetBB2
  2. ViT  

## 3. Criando um extrator de caracteristicas EffNetB2

extrator de caracteristicas = Um termo para "transer learning", aonde o modelo tem suas camadas bases congeladas e saida customizada para o problema em questao.

In [7]:
#PEgando os pesos
effnetb2_weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT

#PEgando os transforms
efnetb2_transforms = effnetb2_weights.transforms()

#Instanciando um model effnet com os pesos DEFAULT
effnetb2 = torchvision.models.efficientnet_b2(weights=effnetb2_weights)

#COngelando as camadas baes
for param in effnetb2.parameters():
 param.requires_grad=False

Downloading: "https://download.pytorch.org/models/efficientnet_b2_rwightman-c35c1473.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b2_rwightman-c35c1473.pth
100%|██████████| 35.2M/35.2M [00:02<00:00, 17.9MB/s]


In [8]:
from torchinfo import summary

 # Print EffNetB2 model summary (uncomment for full output)
saidas = summary(effnetb2,
         input_size=(1, 3, 224, 224),
         col_names=["input_size", "output_size", "num_params", "trainable"],
         col_width=20,
         row_settings=["var_names"])
#saidas

In [9]:
effnetb2.classifier

Sequential(
  (0): Dropout(p=0.3, inplace=True)
  (1): Linear(in_features=1408, out_features=1000, bias=True)
)

In [10]:
set_seeds()
effnetb2.classifier = nn.Sequential(nn.Dropout(p=0.3, inplace=True),
                                    nn.Linear(in_features=1408, out_features=3))

In [11]:
 # Print EffNetB2 model summary (uncomment for full output)
saidas2 = summary(effnetb2,
         input_size=(1, 3, 224, 224),
         col_names=["input_size", "output_size", "num_params", "trainable"],
         col_width=20,
         row_settings=["var_names"])
saidas2

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [1, 3, 224, 224]     [1, 3]               --                   Partial
├─Sequential (features)                                      [1, 3, 224, 224]     [1, 1408, 7, 7]      --                   False
│    └─Conv2dNormActivation (0)                              [1, 3, 224, 224]     [1, 32, 112, 112]    --                   False
│    │    └─Conv2d (0)                                       [1, 3, 224, 224]     [1, 32, 112, 112]    (864)                False
│    │    └─BatchNorm2d (1)                                  [1, 32, 112, 112]    [1, 32, 112, 112]    (64)                 False
│    │    └─SiLU (2)                                         [1, 32, 112, 112]    [1, 32, 112, 112]    --                   --
│    └─Sequential (1)                                        [1, 32, 112, 112]    [1, 1

### Criando uma funcao para criar um modelo de extracao de caracteristicas EffNetB2

In [12]:
def create_effnetb2(num_classes:int = 3,
                    seed:int=3):

  # Pesos, transforms, e instanciando o modelo
  weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
  transforms = weights.transforms()
  model = torchvision.models.efficientnet_b2(weights=weights)

  #COngelando as camdas base
  for param in model.parameters():
    requires_grad=False

  #AJustando o classifier para o problema em questao
  torch.manual_seed(seed)
  model.classifier = nn.Sequential(
      nn.Dropout(p=0.3, inplace=True),
      nn.Linear(in_features=1408, out_features=3))

  return model, transforms

In [13]:
effnetb2, effnetb2_transforms = create_effnetb2(num_classes=3, seed=42)

In [14]:
effnetb2_transforms

ImageClassification(
    crop_size=[288]
    resize_size=[288]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

In [15]:
summary(effnetb2,
        input_size=(1, 3, 224, 224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [1, 3, 224, 224]     [1, 3]               --                   True
├─Sequential (features)                                      [1, 3, 224, 224]     [1, 1408, 7, 7]      --                   True
│    └─Conv2dNormActivation (0)                              [1, 3, 224, 224]     [1, 32, 112, 112]    --                   True
│    │    └─Conv2d (0)                                       [1, 3, 224, 224]     [1, 32, 112, 112]    864                  True
│    │    └─BatchNorm2d (1)                                  [1, 32, 112, 112]    [1, 32, 112, 112]    64                   True
│    │    └─SiLU (2)                                         [1, 32, 112, 112]    [1, 32, 112, 112]    --                   --
│    └─Sequential (1)                                        [1, 32, 112, 112]    [1, 16, 112,

#COnfigurando Dataloaders pra o EffNetB2


In [16]:
from going_modular.going_modular import data_setup

train_datalaoder_effnetb2, test_dataloader_effnetb2, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                                                 test_dir=test_dir,
                                                                                                 transform=effnetb2_transforms,
                                                                                                 batch_size=32)

In [17]:
len(train_datalaoder_effnetb2), len(test_dataloader_effnetb2), class_names

(15, 5, ['pizza', 'steak', 'sushi'])

### Treinado o extrator de caracteristicas EffNetB2

In [ ]:
from going_modular.going_modular import engine
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(effnetb2.parameters(), lr=1e-3)

modelo = engine.train(model=effnetb2,
                      train_dataloader=train_datalaoder_effnetb2,
                      test_dataloader=test_dataloader_effnetb2,
                      optimizer=optimizer,
                      loss_fn=loss_fn,
                      epochs=10,
                      device=device)

  0%|          | 0/10 [00:00<?, ?it/s]

### 3.4 Inspecionando os curvas da EffNetB2


In [ ]:
from helper_functions import plot_loss_curves
plot_loss_curves(modelo)

### 3.5 Salvando o modelo EffNetB2


In [ ]:
from going_modular.going_modular import utils

utils.save_model(model=effnetb2,
                 target_dir='models',
                 model_name="09_pretrained_effnetb2_feature_extractor_pizza_steak_sushi_20_percent.pth")

### 3.6 Descobrindo o tamanho do arquivo do modelo

Pq isso e importante?

Pq estamos lancando esse modelo pra ser usado em um app/mobile que pode ter limitacoes de recurcos computacionais. Assim se o modelo for muit grande (para os dispositivos em questao) ele pode nao rodar/ ter problemas ao rodar.


In [ ]:
from pathlib import Path

#Pegando o tamanho em bytes e convertendo para mega
pretrained_effnetb2_model_size = Path("models/09_pretrained_effnetb2_feature_extractor_pizza_steak_sushi_20_percent.pth").stat().st_size // (1024*1024)
print(f"Tamanho do modelo {round(pretrained_effnetb2_model_size, 2)}MB")

### 3.7 Estatisticas do extrator de caracteristicas da EffNetB2

In [ ]:
#Contagem da quantidade de parametros
effnetb2_total_params = sum(torch.numel(param) for param in effnetb2.parameters())
effnetb2_total_params

In [ ]:
#Criando um dicionarios com os resultados do treinamento do modelo para posterior comparacao
effnetb2_stats = {"test_loss": modelo["test_loss"][-1],
                  "test_acc": modelo["test_acc"][-1],
                  "number_of_parameters": effnetb2_total_params,
                  "model_size (MB)": pretrained_effnetb2_model_size}
effnetb2_stats

## 4. Criando um extrator de caracteristicas usando o ViT

In [ ]:
vit_weights = torchvision.models.ViT_B_16_Weights.DEFAULT

vit_transforms = vit_weights.transforms()

vit = torchvision.models.vit_b_16(weights=vit_weights)

In [ ]:
for param in vit.parameters():
  requires_grad=False

In [ ]:
summary(vit,
        input_size=(1, 3, 224, 224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

In [ ]:
vit.heads

In [ ]:
vit.heads = nn.Sequential(
    nn.Linear(in_features=768, out_features=3)
)

In [ ]:
summary(vit,
        input_size=(1, 3, 224, 224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

In [ ]:
def create_vit(out_features:int=3, seed:int=42):
  vit_weights = torchvision.models.ViT_B_16_Weights.DEFAULT

  vit_transforms = vit_weights.transforms()

  vit = torchvision.models.vit_b_16(weights=vit_weights)

  for param in vit.parameters():
    param.requires_grad=False

  vit.heads = nn.Sequential(
    nn.Linear(in_features=768, out_features=3)
    )

  return vit, vit_transforms

In [ ]:
vit, vit_transform = create_vit(out_features=3,
           seed=42)
vit_transform

In [ ]:
summary(vit,
        input_size=(1, 3, 224, 224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

### 4.1 Criando os DataLoaders para a ViT

In [ ]:
train_vit_dataloader, test_vit_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                                       test_dir=test_dir,
                                                                                       transform=vit_transform,
                                                                                       batch_size=32)
len(train_vit_dataloader), len(test_vit_dataloader), class_names

### 4.2 Treinando a ViT

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vit.parameters(), lr=1e-3)


In [ ]:
vit_results = engine.train(model=vit,
                           train_dataloader=train_vit_dataloader,
                           test_dataloader=test_vit_dataloader,
                           optimizer=optimizer,
                           loss_fn=loss_fn,
                           epochs=10,
                           device=device)

### 4.3 Plotando as curvas do treinamento

In [ ]:
plot_loss_curves(vit_results)

### 4.4 Salvando a ViT

In [ ]:
utils.save_model(model=vit,
                 target_dir='models',
                model_name="09_pretrained_vit_feature_extrator_pizza_steak_sushi_20_percent.pth")

### 4.5 Checando o tamanho do modelo ViT

In [ ]:
tamanho_modelo= Path("models/09_pretrained_vit_feature_extrator_pizza_steak_sushi_20_percent.pth").stat().st_size // (1024*1024)
print(f" size: {tamanho_modelo} mb")


### 4.6 Estatisticas do modelo ViT

In [ ]:
#Contagem da quantidade de parametros
vit_total_params = sum(torch.numel(param) for param in vit.parameters())
vit_total_params

In [ ]:
vit_stats = {"test_loss" : vit_results["test_loss"][-1],
             "test_acc" : vit_results["test_acc"][-1],
             "number_of_parameters" : vit_total_params,
             "model_size (MB)": tamanho_modelo
             }
vit_stats

## 5. Realizando predicoes com os modelos treinados e conometrando

Objetivos
  * 95%+ Acc
  * 30+ FPS

Criterios do teste:

1. Passar pelas imagens de teste
2. Calcular quanto tempo cada modelo leva para realizar a predicao na imagem

In [ ]:
from pathlib import Path

#PPEgando todos os aminhos dos dadoss de teste
test_data_paths = list(Path(test_dir).glob("*/*.jpg"))
test_data_paths[:5]

### 5.1 Criando a funcao para fazer teste em todo dataset

* `pred_and_store()`

1. Criar um funcao que receba uma lista de caminhos, um modelo pytorch, um transforms, lista das classes alvo, e um dispositivo.
2. Criar uma lista vazia, que ira receber todas as predicoes
3. PErcorrer sobre os caminhos dos dados (os proximos passos ocorreram dentro dest loop).
4. Criar um dicionario vazio para cada amostra (As stats de predicao irao aqui)
5. Pegar o caminho da amostra e a classe verdadeira do filepath
6. Comecar o tempo da predicao
7.  Abrir a imagem usando `PIL.Image.open(path)`.
8. Transformar a imagem para estar apta ao modelo.
9. Preparar o modelo para inferencia, mandar para o dispositivo alvo e acionar o modo `eval()`.
10. Ligar o torch.inference_mode() e passar a imagem alvo transformada para o modelo e performar o forward pass + calculo da pred prob + pred class.
11. Adicionar a pred prob + pred class ao dicionario do passo 4.
12. Encerrrar o timer, e adicionar o tempo decorrido ao dicionario.
13. Verificar se as classes preditas sao as verdadeiras.
14. Adiconar ao fim da lista as predicoes conforme vao ocorrendo
15. Devolver a list de dicionarios de predicoes.


In [ ]:
import pathlib
import torch

from PIL import Image
from timeit import default_timer as timer
from tqdm.auto import tqdm
from typing import Dict, List

# 1.
def pred_and_store(paths: List[pathlib.Path],
                   model: torch.nn.Module,
                   transform: torchvision.transforms,
                   class_names: List[str],
                   device: str = "cuda" if torch.cuda.is_available() else "cpu") -> List[Dict]:

  #2.
  pred_list = []

  #3.
  for path in tqdm(paths):

    # 4.
    pred_dict = {}

    # 5.
    pred_dict["image_path"] = path
    class_names = path.parent.stem
    pred_dict["class_names"] = class_names

    # 6.
    start_time = timer()

    # 7.
    img = Image.open(path)

    # 8.
    transformed_image = transform(img).unsqueeze(0).to(device)

    # 9.
    model = model.to(device)
    model.eval()

    # 10.
    with torch.inference_mode():
      pred_logit = model(transformed_image)
      pred_prob = torch.softmax(pred_logit, dim=1)
      pred_label = torch.argmax(pred_prob, dim=1)
      pred_class = class_names[pred_label.cpu()]

      # 11.
      pred_dict["pred_prob"] = round(pred_prob.unsqueeze(0).max().cpu().item(), 4)
      pred_dict["pred_class"] = pred_class

      # 12.
      end_time = timer()
      pred_dict["time_for_pred"] = round(end_time - start_time, 4)

    # 13.
    pred_dict["correct"] = class_names == pred_class

    # 14.
    pred_list.append(pred_dict)
  # 15. devolve a lista de dicionarios com as predicoes
  return pred_list

In [ ]:
import pathlib
import torch

from PIL import Image
from timeit import default_timer as timer
from tqdm.auto import tqdm
from typing import List, Dict

# 1. Create a function to return a list of dictionaries with sample, truth label, prediction, prediction probability and prediction time
def pred_and_store(paths: List[pathlib.Path],
                   model: torch.nn.Module,
                   transform: torchvision.transforms,
                   class_names: List[str],
                   device: str = "cuda" if torch.cuda.is_available() else "cpu") -> List[Dict]:

    # 2. Create an empty list to store prediction dictionaries
    pred_list = []

    # 3. Loop through target paths
    for path in tqdm(paths):

        # 4. Create empty dictionary to store prediction information for each sample
        pred_dict = {}

        # 5. Get the sample path and ground truth class name
        pred_dict["image_path"] = path
        class_name = path.parent.stem
        pred_dict["class_name"] = class_name

        # 6. Start the prediction timer
        start_time = timer()

        # 7. Open image path
        img = Image.open(path)

        # 8. Transform the image, add batch dimension and put image on target device
        transformed_image = transform(img).unsqueeze(0).to(device)

        # 9. Prepare model for inference by sending it to target device and turning on eval() mode
        model.to(device)
        model.eval()

        # 10. Get prediction probability, predicition label and prediction class
        with torch.inference_mode():
            pred_logit = model(transformed_image) # perform inference on target sample
            pred_prob = torch.softmax(pred_logit, dim=1) # turn logits into prediction probabilities
            pred_label = torch.argmax(pred_prob, dim=1) # turn prediction probabilities into prediction label
            pred_class = class_names[pred_label.cpu()] # hardcode prediction class to be on CPU

            # 11. Make sure things in the dictionary are on CPU (required for inspecting predictions later on)
            pred_dict["pred_prob"] = round(pred_prob.unsqueeze(0).max().cpu().item(), 4)
            pred_dict["pred_class"] = pred_class

            # 12. End the timer and calculate time per pred
            end_time = timer()
            pred_dict["time_for_pred"] = round(end_time-start_time, 4)

        # 13. Does the pred match the true label?
        pred_dict["correct"] = class_name == pred_class

        # 14. Add the dictionary to the list of preds
        pred_list.append(pred_dict)

    # 15. Return list of prediction dictionaries
    return pred_list

### 5.2 Realizando as predicoes e conometrando-as

Usando a `pred_and_store()`

Pontos chaves:
1. O dispositivo sera CPU, configurada manualmente (Pq nao se tem certeza que sempre havera uma GPU quando o medelo entrar em deploy)
2. Transforms - os modelo tem que estarem realizando as predicoes com os dados de teste que foram transformados para os repectivos modelos.

#### EffNetB2

In [ ]:
effnetb2_test_pred_dicts = pred_and_store(paths=test_data_paths,
                                          model=effnetb2,
                                          transform=effnetb2_transforms,
                                          class_names=class_names,
                                          device="cpu")

In [ ]:
effnetb2_test_pred_dicts[:2]


In [ ]:
#TRansformando o test_pred_dicts em um DataFrame
import pandas as pd
effnetb2_test_pred_df = pd.DataFrame(effnetb2_test_pred_dicts)
effnetb2_test_pred_df.head()

In [ ]:
#ChecaNDO O NUMERO DE PREVISOES CORRETAS
effnetb2_test_pred_df.correct.value_counts()

In [ ]:
#Tempo medio de predicao
effnetb2_average_time_per_pred= round(effnetb2_test_pred_df.time_for_pred.mean(), 4)
print(f"Tempo medio de predicao da EffNetB2: {effnetb2_average_time_per_pred}")


In [ ]:
effnetb2_stats["time_per_pred_cpu"] = effnetb2_average_time_per_pred
effnetb2_stats

#### ViT

In [ ]:
vit_test_pred_dicts = pred_and_store(paths=test_data_paths,
                                          model=vit,
                                          transform=vit_transforms,
                                          class_names=class_names,
                                          device="cpu")
vit_test_pred_dicts[:2]

In [ ]:
import pandas as pd
vit_test_pred_df = pd.DataFrame(vit_test_pred_dicts)
vit_test_pred_df.head()

In [ ]:
#ChecaNDO O NUMERO DE PREVISOES CORRETAS
vit_test_pred_df.correct.value_counts()

In [ ]:
#Tempo medio de predicao
vit_average_time_per_pred= round(vit_test_pred_df.time_for_pred.mean(), 4)
print(f"Tempo medio de predicao da ViT: {vit_average_time_per_pred} ")

In [ ]:
# Add average prediction time for ViT model on CPU
vit_stats["time_per_pred_cpu"] = vit_average_time_per_pred
vit_stats

In [ ]:
effnetb2_stats

## 6. Comparando os resultados dos modelos

In [ ]:
#transformando os dict em dataframws
df = pd.DataFrame([effnetb2_stats, vit_stats])

#Add uma coluna de nome
df["model"] = ['EffNetB2', "ViT"]

#Converte acc em porcentagem
df["test_acc"] = round(df["test_acc"] * 100, 2)

In [ ]:
df

Qual modelo eh o melhor?
* test_loss (menos e melhor) -ViT
* test_acc (mais e melhor) -ViT
* number_of_parameters (geralmente menor e maior*) - se um modelo tem mais parametros, geralmente demora mais para computr
  * *as vezes modelos com mais parametros ainda conseguem computar mais rapido
* model_size (MB) - EffNetB2 (PAra o nosso caso que o deploy sera para mobile, geralmente menor eh melhor)
* time_per_pred_cpu (menos e melhor, sera altamente dependendo do hardware que esta rodando

Os dois modelos falharam em atingir o objetivo de 30+FPS... Com tudo se pode so tenatr e usar o EffNetB2 e ver como ele se sai

In [ ]:
#Comprando ViT com EffNetB2
pd.DataFrame(data=(df.set_index("model").loc["ViT"] / df.set_index("model").loc["EffNetB2"]),
             columns=["ViT to EffNetB2 ratios"]).T

### 6.1 Visualizando o tradeoff Velocidade X Performace

PAra comparar os dois features extrator de maneira visual:

1. Criar um scatter plot usando o datafrade para comprar a acc de teste e o tempo de predicao entre EffNetB2 X ViT.
2. Add titulos e rotulos pra grafico ficar mais legivel.
3. Anotar os amostras no grafico
4. Criar a legende para o grafico com base no tamanho dos modelos (MB)

In [ ]:
# 1. Criar grafico para comparacao do DataFrame
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(12, 8))
scatter = ax.scatter(data=df,
                     x="time_per_pred_cpu",
                     y="test_acc",
                     c=["blue", "orange"],
                     s="model_size (MB)")

# 2. Titulos e labels
ax.set_title("FoodVision Mini", fontsize=18)
ax.set_xlabel("TEmpo de predicao por imagem em segundos", fontsize=14)
ax.set_ylabel("TEste Acc", fontsize=14)
ax.tick_params(axis="both", labelsize=12)
ax.grid(True)

# 3.
for index, row in df.iterrows():
  ax.annotate(text=row["model"],
              xy=(row["time_per_pred_cpu"]+0.0006, row["test_acc"]+0.03),
              size=12)

# 4. Criando as legendas baseada nos tamanho dos modelos
handles, labels = scatter.legend_elements(prop="sizes", alpha=0.5)
model_size_legend = ax.legend(handles,
                              labels,
                              loc="lower right",
                              fontsize=12)


plt.savefig("09-foodvision-mini-inference-speed-vs-performance.jpg")

plt.show();

## 7. Dando vida ao FoodVision Mini Usando uma demo do Gradio

EffNetB2 foi escolhido com base nos criterio

In [ ]:
# Import/install Gradio
try:
    import gradio as gr
except:
    !pip -q install gradio
    import gradio as gr

print(f"Gradio version: {gr.__version__}")

### 7.1 Gradio Overview

PQ usar? Pra criar um mvp do modelo que possa ser testado por outras pessoas.



```
inputs -> fucntion/model -> outputs
```



### 7.2  Criando uma funcao para mapear nossas entradas e saidas

In [ ]:
effnetb2 = effnetb2.to("cpu")

#Checando se foi para o dispositivo
next(iter(effnetb2.parameters())).device

A funcao `predict()` ira fazer:



```
Imagem de comida -> Modelo (EffNetB2) -> [Classe da comida e tempo de predicao]
```



In [ ]:
from typing import Dict, Tuple

def predict(img) -> Tuple[Dict, float]:

  #Comeca o time
  start_time = timer()

  #Transforma a imagem para o padrao EffNetB2
  img = effnetb2_transforms(img).unsqueeze(0) #unqueeze = add uma dimensao batch na posicao 0

  #Colocando o model em eval
  effnetb2.eval()
  with torch.inference_mode():

    #Pasando a imagem transformada pelo modelo de devolvendo os logist em pred prob
    pred_prob = torch.softmax(effnetb2(img), dim=1)

  # Criando a label de predicao e o dicionario
  pred_labels_and_probs = {class_names[i]: float(pred_probs[0][i]) for i in range(len(class_names))}

  #Calculando o tempo de predicoa
  end_time = timer()
  pred_time = round(end_time - start_time, 4)

  return pred_labels_and_probs, pred_time

In [ ]:
import random
from PIL import Image

#Pega em lista todos os caminhos da pasta test
test_data_path = list(Path(test_dir).glob("*/*.jpg"))
test_data_path

#Selecionando a imagem de test aleatoriamente
random_image_path = random.sample(test_data_paths, k=1)[0]
random_image_path

#Open the target image
image = Image.open(random_image_path)
print(f"[INFO] Predizendo a imagem de caminho: {random_image_path}\n")

#Prevendo e mostrando o dict e tempo
pred_dict, pred_time = predict(img=image)
print(pred_dict)
print(pred_time)

### 7.3 Criando uma lista de exemplos de Imagens

In [ ]:
#Criando a lista de exemplos
exemple_list = [[str(filepath)] for filepath in random.sample(test_data_paths, k=3)]
exemple_list